In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [17]:
df = pd.read_csv("data/주택종류별+주택-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20240314151334.csv")

In [18]:
df

,동별(1),동별(2),동별(3),2020,2020.1,2020.2,2020.3,2020.4,2020.5,2020.6,2020.7,2020.8
0,동별(1),동별(2),동별(3),종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수,종류별 주택수
1,동별(1),동별(2),동별(3),소계,단독주택,단독주택,단독주택,단독주택,아파트,연립주택,다세대주택,비거주용건물내주택
2,동별(1),동별(2),동별(3),소계,소계,일반단독주택,다가구주택,영업겸용주택,소계,소계,소계,소계
3,서울시,소계,소계,3015371,307075,64911,190399,51765,1772670,110562,796066,28998
4,서울시,종로구,소계,46859,12259,6572,4070,1617,15688,5460,12540,912
...,...,...,...,...,...,...,...,...,...,...,...,...
449,서울시,강동구,둔촌1동,5,5,X,X,X,X,X,X,X
450,서울시,강동구,둔촌2동,7962,543,31,377,135,5011,154,2187,67
451,서울시,강동구,암사1동,10089,1139,20,889,230,2790,247,5829,84
452,서울시,강동구,천호2동,9362,1488,103,1134,251,2641,189,4901,143


In [19]:
# 인덱스 0, 1, 2 제거
df = df.drop(index=[0, 1, 2])

In [20]:
# '소계' 행 제거
df = df[df['동별(3)'] != '소계']

In [21]:
# 동별(1), 동별(2) 열 제거
df = df.drop(columns = ['동별(1)', '동별(2)', '2020', '2020.1', '2020.5'])

In [22]:
df

,동별(3),2020.2,2020.3,2020.4,2020.6,2020.7,2020.8
5,사직동,467,108,75,78,411,48
6,삼청동,414,134,96,19,37,30
7,부암동,750,214,69,903,976,37
8,평창동,1088,314,151,2186,1139,71
9,무악동,14,9,25,55,46,8
...,...,...,...,...,...,...,...
449,둔촌1동,X,X,X,X,X,X
450,둔촌2동,31,377,135,154,2187,67
451,암사1동,20,889,230,247,5829,84
452,천호2동,103,1134,251,189,4901,143


In [23]:
# 열 이름 변경
df = df.rename(columns={
    '동별(3)': '동',
    '2020.2': '일반단독주택',
    '2020.3': '다가구주택',
    '2020.4': '영업겸용주택',
    '2020.6': '연립주택',
    '2020.7': '다세대주택',
    '2020.8': '비거주용건물내주택'
})
df

,동,일반단독주택,다가구주택,영업겸용주택,연립주택,다세대주택,비거주용건물내주택
5,사직동,467,108,75,78,411,48
6,삼청동,414,134,96,19,37,30
7,부암동,750,214,69,903,976,37
8,평창동,1088,314,151,2186,1139,71
9,무악동,14,9,25,55,46,8
...,...,...,...,...,...,...,...
449,둔촌1동,X,X,X,X,X,X
450,둔촌2동,31,377,135,154,2187,67
451,암사1동,20,889,230,247,5829,84
452,천호2동,103,1134,251,189,4901,143


424